In [17]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
import tqdm
import PIL
import numpy as np

In [5]:
drop_rate = 0

train_val_test_split = (0.7,0.2,0.1)
batch_size = 96
num_workers=6
drop_rate = 0.23
train_dir = '../data/train'

In [6]:
# Pytorch Convolutional Neural Network Model Architecture
class CatAndDogConvNet(nn.Module):

    def __init__(self):
        super().__init__()

        # onvolutional layers (3,16,32)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 64 * 3 * 3, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, X):

        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

In [7]:
teacher_model = torch.load('../models/densenet201.pt')


student_model = torch.load('../models/simple_cnn.pt')


In [10]:
train_files = random.sample(os.listdir(train_dir),k = 24000)
# Only select jpg files
train_files = [f for f in train_files if '.jpg' in f]


In [11]:
# Defining augmentations

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [12]:
# Creating dataset

class DogsVsCatsDataset(Dataset):
    def __init__(self, file_list, dir, mode='train', transform = val_transform):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = PIL.Image.open(os.path.join(self.dir, self.file_list[idx]))
        img = self.transform(img)
        img = np.array(img)
        if 'dog' in self.file_list[idx]:
            self.label = 1
        else:
            self.label = 0
        return img.astype('float32'), self.label


train_files, test_files = train_test_split(train_files, 
                                    test_size=train_val_test_split[2], 
                                    random_state=42
                                    )
train_files, valid_files = train_test_split(train_files,
                                    test_size=train_val_test_split[1]/train_val_test_split[0], 
                                    random_state=42
                                    )

TrainDataSet = DogsVsCatsDataset(train_files, dir = train_dir, mode='train', transform = train_transform)
TrainDataLoader = DataLoader(TrainDataSet, batch_size = batch_size, shuffle=True, num_workers=num_workers)

ValidDataSet = DogsVsCatsDataset(valid_files, dir = train_dir, mode='valid')
ValidDataLoader = DataLoader(ValidDataSet, batch_size = batch_size, shuffle=False, num_workers=num_workers)

TestDataSet = DogsVsCatsDataset(test_files, dir = train_dir, mode='test')
TestDataLoader = DataLoader(TestDataSet, batch_size = batch_size, shuffle=False, num_workers=num_workers)

In [13]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(teacher_model.parameters(), lr=lr, amsgrad=True)
teacher_model = teacher_model.to(device)

student_model = student_model.to(device)

NameError: name 'lr' is not defined

In [14]:
test_acc = 0
itr = 1
tot_itr = len(TestDataLoader)
for samples, labels in tqdm.tqdm(TestDataLoader, desc = "Testing", unit = " Iterations"):
        with torch.no_grad():
            samples, labels = samples.to(device), labels.to(device)
            output = teacher_model(samples)
            loss = criterion(output, labels)
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            test_acc += torch.mean(correct.float())
            torch.cuda.empty_cache()
            itr += 1
print('-----------------------------> Testing Accuracy: {:.1f} %'.format(test_acc/tot_itr*100))

Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:18<00:00,  1.32 Iterations/s]

-----------------------------> Testing Accuracy: 91.9 %


In [18]:
test_acc = 0
itr = 1
tot_itr = len(TestDataLoader)
for samples, labels in tqdm.tqdm(TestDataLoader, desc = "Testing", unit = " Iterations"):
        with torch.no_grad():
            samples, labels = samples.to(device), labels.to(device)
            output = student_model(samples)
            loss = criterion(output, labels)
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            test_acc += torch.mean(correct.float())
            torch.cuda.empty_cache()
            itr += 1
print('-----------------------------> Testing Accuracy: {:.1f} %'.format(test_acc/tot_itr*100))

Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:17<00:00,  1.39 Iterations/s]

-----------------------------> Testing Accuracy: 65.7 %
